In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import hashlib
import re
from cryptography.fernet import Fernet

# Загрузка данных
file_path = '/content/drive/MyDrive/Laptop_price.csv'  # Замените на путь к файлу
try:
    df = pd.read_csv(file_path)  # Читаем CSV-файл и загружаем его в DataFrame
except Exception as e:
    print(f"Ошибка загрузки файла: {e}")

# 1. Проверка CSV на уязвимости (CSV Injection)
# Дополнительно проверим на пробелы перед опасными символами и SQL-комментарии
def check_csv_injection(df):
    dangerous_chars = ('=', '+', '-', '@')
    comment_pattern = '--'  # Комментарий в SQL-запросах

    for col in df.select_dtypes(include=['object']).columns:
        # Проверка на начинающиеся с пробелом перед символом или сами символы
        starts_with_dangerous = df[col].astype(str).apply(
            lambda x: any(x.strip().startswith(char) for char in dangerous_chars)
        ).any()

        # Проверка на SQL-комментарии
        contains_sql_comment = df[col].astype(str).apply(lambda x: comment_pattern in x).any()

        if starts_with_dangerous or contains_sql_comment:
            print(f"Обнаружены потенциальные CSV-инъекции или SQL-комментарии в столбце {col}!")
        else:
            print(f"Столбец {col} безопасен.")

check_csv_injection(df)

# 2. Фильтрация данных от SQL-инъекций и XSS-атак
# SQL-инъекции используются для взлома баз данных, а XSS – для атак на пользователей
# Добавим проверку на SQL-комментарии
def clean_input(value):
    sql_keywords = ["SELECT", "DROP", "DELETE", "INSERT", "UPDATE", "ALTER", "UNION", "--"]  # Добавлены SQL-комментарии
    xss_patterns = [r'<script.*?>.*?</script>', r'javascript:.*', r'onerror=.*']

    for keyword in sql_keywords:
        if keyword.lower() in value.lower():
            return "[BLOCKED]"

    for pattern in xss_patterns:
        if re.search(pattern, value):
            return "[BLOCKED]"

    return value

# Применяем чистку ко всем строковым полям
df = df.applymap(lambda x: clean_input(str(x)) if isinstance(x, str) else x)
print("Фильтрация данных завершена.")

# 3. Хеширование столбца с ценами (SHA-256)
# Хеширование используется для защиты конфиденциальных данных, таких как пароли
# В данном случае мы хешируем цену ноутбука, чтобы скрыть оригинальное значение
def hash_price(price):
    return hashlib.sha256(str(price).encode()).hexdigest()

df['Price_Hashed'] = df['Price'].apply(hash_price)
print("Столбец с хешированными ценами добавлен.")

# 4. Шифрование данных (например, цены ноутбука)
# Шифрование позволяет скрыть данные, но при этом их можно расшифровать при наличии ключа
# Генерируем ключ и шифруем цену ноутбука
key = Fernet.generate_key()
cipher = Fernet(key)

def encrypt_price(price):
    return cipher.encrypt(str(price).encode()).decode()

def decrypt_price(encrypted_price):
    return cipher.decrypt(encrypted_price.encode()).decode()

df['Price_Encrypted'] = df['Price'].apply(encrypt_price)
print("Столбец с зашифрованными ценами добавлен.")

# 5. Шифрование столбца RAM_Size
# Добавляем новую колонку с зашифрованной RAM_Size
def encrypt_ram_size(ram_size):
    return cipher.encrypt(str(ram_size).encode()).decode()

df['RAM_Size_Encrypted'] = df['RAM_Size'].apply(encrypt_ram_size)
print("Столбец с зашифрованной RAM_Size добавлен.")

# 6. Функция расшифровки RAM_Size
# Декодируем первые 5 записей RAM_Size
def decrypt_ram_size(encrypted_ram_size):
    return cipher.decrypt(encrypted_ram_size.encode()).decode()

first_five_decrypted_ram_sizes = df['RAM_Size_Encrypted'].head(5).apply(decrypt_ram_size)
for idx, dec_ram_size in first_five_decrypted_ram_sizes.items():
    print(f"Расшифрованная RAM_Size №{idx}: {dec_ram_size}")



Столбец Brand безопасен.
Фильтрация данных завершена.
Столбец с хешированными ценами добавлен.
Столбец с зашифрованными ценами добавлен.
Столбец с зашифрованной RAM_Size добавлен.
Расшифрованная RAM_Size №0: 16
Расшифрованная RAM_Size №1: 4
Расшифрованная RAM_Size №2: 4
Расшифрованная RAM_Size №3: 16
Расшифрованная RAM_Size №4: 32


<ipython-input-43-0eec5e598414>:53: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: clean_input(str(x)) if isinstance(x, str) else x)


In [ ]:
# Сохранение обработанных данных
output_path = "/content/drive/MyDrive/Laptop_price_secure.csv"
df.to_csv(output_path, index=False)
print(f"Обработанный файл сохранен: {output_path}")

Обработанный файл сохранен: /content/drive/MyDrive/Laptop_price_secure.csv


In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)      # Отображаем ВСЕ строки
pd.set_option('display.max_columns', None)   # Отображаем ВСЕ столбцы
pd.set_option('display.width', None)         # Без ограничения ширины экрана
pd.set_option('display.expand_frame_repr', False)  # Запрещаем перенос длинных строк

# Наш DataFrame
df = pd.read_csv('Laptop_price_secure.csv')

# Вывод всей таблицы
df

Output hidden; open in https://colab.research.google.com to view.